### Data Structures

Our applications and code samples are based on a simple data structure for parametric surfaces and curves. This notebook introduces the data structure and basic operations.

In [ ]:
import matplotlib.pyplot as plt
import torch
import trimesh

device = torch.device('cpu')

The data structures are very similar to indexed face sets for meshes. They are designed to hold a set of parametric curves/surfaces, for example three cubic Bézier curves.

The control point positions are stored in an array `V`; the array `F` holds the indices into `V` for each individual curve/surface.

This example shows the data structure for curves:

In [ ]:
from drpg import ParametricCurves

# Create a data structure that holds cubic Bézier curves in 2D
degree    = 3
dimension = 2
curves = ParametricCurves(n=degree+1, d=dimension, device=device)

# Add three random curves
curves.add(torch.rand(3, degree+1, 2))

print(f"{curves.V.shape=}", f"{curves.V[0]=}", f"{curves.V.dtype=}")
print(f"{curves.F.shape=}", f"{curves.F[0]=}", f"{curves.F.dtype=}")

# Get the control point positions of all curves
C = curves.V[curves.F]
print(f"{C.shape=}")

# Display the control points
fig, ax = plt.subplots(1, 1, figsize=(4, 4))
for i in range(len(curves.F)):
    c = C[i]
    ax.plot(c[:, 0].cpu(), c[:, 1].cpu())
    ax.scatter(c[:, 0].cpu(), c[:, 1].cpu(), label=f"Control points {i}")
ax.legend()

The data structure for surface patches is very similar:

In [ ]:
from drpg import ParametricPatches, create_quadratic_grid, generate_grid_faces

import numpy as np

# Create a data structure that holds biquadratic Bézier patches in 3D
degree = 2
patches = ParametricPatches(n=degree+1, device=device)

# Add a random patch and a patch with control point following a quadratic function
patches.add(torch.rand(1, degree+1, degree+1, 3))
patches.add(create_quadratic_grid(degree+1, degree+1, a=0.2, b=0.3, position=[0, 0, 0], normal=[0, 1, 0], tangent=[1, 0, 0], scale=0.5, device=device))

print(f"{patches.V.shape=}", f"{patches.V[0]=}", f"{patches.V.dtype=}")
print(f"{patches.F.shape=}", f"{patches.F[0]=}", f"{patches.F.dtype=}")

# Get the control point positions of all patches
C = patches.V[patches.F]
print(f"{C.shape=}")

# Display the control meshes
scene = trimesh.Scene()
for i in range(len(patches.F)):
    c = C[i]

    # Generate a unique color for this patch
    color  = (255*(np.random.rand(3)*0.5 + 0.5)).astype(np.uint8)

    # Create geometry for the control mesh
    f = generate_grid_faces(patches.n, patches.m)
    mesh = trimesh.Trimesh(c.reshape(-1, 3), f)
    mesh.visual = trimesh.visual.ColorVisuals(vertex_colors=np.tile(color, reps=(mesh.vertices.shape[0], 1)))

    # Create geometry for the control points
    points = [trimesh.primitives.Sphere(radius=0.01, center=pt).to_mesh() for pt in c.reshape(-1, 3)]
    for point in points:
        point.visual = trimesh.visual.ColorVisuals(vertex_colors=np.tile(color, reps=(point.vertices.shape[0], 1)))

    scene.add_geometry(mesh)
    scene.add_geometry(points)
scene.show()